In [ ]:
# necessary imports: usually i import separately in the files
# import tensorflow as tf
# import tensorflow_hub as hub
!pip install transformers
# import transformers as tfs
# using bert from tfs i.e. BertTokenizer

In [ ]:
# bring in the files from github sometimes necessary once
! git clone https://github.com/mag389/holbertonschool-machine_learning

In [6]:
# bring in the files from github
# ! git clone https://github.com/mag389/holbertonschool-machine_learning
# install transformers into colab (only have to do the first time)
# !pip install transformers
# begin actual file
import tensorflow as tf
import tensorflow_hub as hub
import transformers
from transformers import BertTokenizer, AutoModelForQuestionAnswering


model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
def question_answer(question, reference):
    """ answers question based on reference document """
    tok = "bert-large-uncased-whole-word-masking-finetuned-squad"
    tokenizer = BertTokenizer.from_pretrained(tok)
    mod = "https://tfhub.dev/see--/bert-uncased-tf2-qa/1"
    # this model hub import is the very slow part
    # model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    question_tokens = tokenizer.tokenize(question)
    reference_tokens = tokenizer.tokenize(reference)

    tokens = ['[CLS]'] + question_tokens + \
        ['[SEP]'] + reference_tokens + ['[SEP]']
    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (1 + len(question_tokens) + 1) + [1] * \
        (len(reference_tokens) + 1)
    input_word_ids, input_mask, input_type_ids = map(
            lambda t: tf.expand_dims(
                tf.convert_to_tensor(t, dtype=tf.int32), 0),
            (input_word_ids, input_mask, input_type_ids))
    outputs = model([input_word_ids, input_mask, input_type_ids])

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    if answer == "":
      answer = None
    return answer
# exit here if you don't wnt to proceed to main
# exit()
# 0 main
with open('/content/holbertonschool-machine_learning/supervised_learning/0x13-qa_bot/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('When are PLDs?', reference))
print(question_answer('What are Mock Interviews?', reference))

on - site days from 9 : 00 am to 3 : 00 pm
None


In [ ]:
# 1 loop
if __name__ == "__main__":
    words = ["exit", "quit", "goodbye", "bye"]
    exited = 0
    while (exited == 0):
        print("Q: ", end="")
        inp = input()
        if inp.lower() in words:
            print("A: Goodbye")
            exited = 1
            break
        else:
            print("A: ")

In [7]:
# 2 

def answer_loop(reference):
    words = ["exit", "quit", "goodbye", "bye"]
    exited = 0
    while (exited == 0):
        print("Q: ", end="")
        inp = input()
        if inp.lower() in words:
            print("A: Goodbye")
            exited = 1
            return
        else:
            answer = question_answer(inp, reference)
            if answer is None or answer == "":
              answer = "Sorry, I do not understand your question."
            print("A: ", end="")
            print(answer)
# main section
with open('/content/holbertonschool-machine_learning/supervised_learning/0x13-qa_bot/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()
answer_loop(reference)

Q: when are plds?
A: on - site days from 9 : 00 am to 3 : 00 pm
Q: what are mock interviews?
A: Sorry, I do not understand your question.
Q: bye
A: Goodbye


In [8]:
# 3 semantic search
# uncomment if not running other sections
import tensorflow as tf
import tensorflow_hub as hub
import transformers
from transformers import BertTokenizer, AutoModelForQuestionAnswering

import glob
import numpy as np
# import tensorflow_hub as hub
# last import for runnign as standalone file


use = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
embed = hub.load(use)
def semantic_search(corpus_path, sentence):
    # use = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
    # embed = hub.load(use)
    embeddings = [sentence]

    for name in glob.glob(corpus_path + "/*"):
        with open(name) as f:
            embeddings += [f.read()]

    # print(type(embeddings))
    # print((embeddings))
    nums = embed(embeddings)
    # print(nums)
    # input()
    corr = np.inner(nums, nums)
    # print(corr[0, 1:])
    # print(np.argmax(corr[0, 1:]))
    # print(corr[0, 1:][np.argmax(corr[0, 1:])])
    am = np.argmax(corr[0, 1:])
    return embeddings[am + 1]

# 3 main section comment out if only creating function for other uses
location = '/content/holbertonschool-machine_learning/supervised_learning/0x13-qa_bot/ZendeskArticles'
print(semantic_search(location, 'When are PLDs?'))
             

PLD Overview
Peer Learning Days (PLDs) are a time for you and your peers to ensure that each of you understands the concepts you've encountered in your projects, as well as a time for everyone to collectively grow in technical, professional, and soft skills. During PLD, you will collaboratively review prior projects with a group of cohort peers.
PLD Basics
PLDs are mandatory on-site days from 9:00 AM to 3:00 PM. If you cannot be present or on time, you must use a PTO. 
No laptops, tablets, or screens are allowed until all tasks have been whiteboarded and understood by the entirety of your group. This time is for whiteboarding, dialogue, and active peer collaboration. After this, you may return to computers with each other to pair or group program. 
Peer Learning Days are not about sharing solutions. This doesn't empower peers with the ability to solve problems themselves! Peer learning is when you share your thought process, whether through conversation, whiteboarding, debugging, or li

In [4]:
# 4 the final qa: 
model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
def question_answer(coprus_path):
    """ answer questions from whole corpus """
    words = ["exit", "quit", "goodbye", "bye"]
    exited = 0
    while (exited == 0):
        print("Q: ", end="")
        inp = input()
        if inp.lower() in words:
            print("A: Goodbye")
            exited = 1
            return
        else:
            refer = semantic_search(coprus_path, inp)
            answer = question_answer1(inp, refer)
            print("A: ", end="")
            print(answer)
# helper functoin
def question_answer1(question, reference):
    """ answers question based on reference document """
    tok = "bert-large-uncased-whole-word-masking-finetuned-squad"
    tokenizer = BertTokenizer.from_pretrained(tok)
    mod = "https://tfhub.dev/see--/bert-uncased-tf2-qa/1"
    # model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    question_tokens = tokenizer.tokenize(question)
    reference_tokens = tokenizer.tokenize(reference)

    tokens = ['[CLS]'] + question_tokens + \
        ['[SEP]'] + reference_tokens + ['[SEP]']
    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (1 + len(question_tokens) + 1) + [1] * \
        (len(reference_tokens) + 1)
    input_word_ids, input_mask, input_type_ids = map(
            lambda t: tf.expand_dims(
                tf.convert_to_tensor(t, dtype=tf.int32), 0),
            (input_word_ids, input_mask, input_type_ids))
    outputs = model([input_word_ids, input_mask, input_type_ids])

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    if answer == "":
        answer = None
    return answer

# and associated main


question_answer('/content/holbertonschool-machine_learning/supervised_learning/0x13-qa_bot/ZendeskArticles')

Q: when are plds?
A: on - site days from 9 : 00 am to 3 : 00 pm
Q: what are mock interviews?
A: help you train for technical interviews
Q: bye
A: Goodbye
